In [ ]:
# iniciamos el método de OneHot Encoder
oh = OneHotEncoder()
transformados = oh.fit_transform(df[['seasons']])
oh_df = pd.DataFrame(transformados.toarray())
#en los apuntes aparece 'get_feature_names', pero no ha funcionado. ha funcionado esto:
oh_df.columns = oh.get_feature_names_out()
oh_df.head()

In [ ]:
# función para convertir los días del mes en dias del año
def convertir_dia_anual (registro):
    dia = pd.Period(registro, freq='H')
    return dia.day_of_year

In [ ]:
df.drop(["dteday"], axis=1, inplace=True)